In [1]:
import pandas as pd
import numpy as np
import duckdb

from sklearn.linear_model import LinearRegression

import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

import mlflow
from mlflow.models import infer_signature

In [2]:
# Caminho do banco
db_path = "../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Define raiz do projeto (ajuste conforme seu ambiente)
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")


# Carrega os dados da camada bronze
df = con.execute("""
    SELECT * FROM feature.previsao_consumo
""").df()

# Treinamento de Modelo

In [3]:
def calcular_metricas(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    nrmse = rmse / np.mean(y_true)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, nrmse, r2

def treinar_modelos_xgb_por_grupo_cv(df, features):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    resultados = []

    grupos = df.groupby(['cluster', 'region'])

    for (cluster, region), grupo_df in grupos:
        print(f"\nTreinando modelo para Cluster {cluster} - Região {region}")

        # Define experimento
        mlflow.set_experiment(f"previsao_cliente_{region}_{cluster}")
        model_name = f"previsao_cliente_{region}_{cluster}"

        X = grupo_df[features]
        y = grupo_df['consumption_kwh']
        tscv = TimeSeriesSplit(n_splits=5)

        maes, rmses, nrmses, r2s = [], [], [], []

        for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = xgb.XGBRegressor(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=3,
                random_state=42,
                n_jobs=-1
            )

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)

            mae, rmse, nrmse, r2 = calcular_metricas(y_val, y_pred)
            maes.append(mae)
            rmses.append(rmse)
            nrmses.append(nrmse)
            r2s.append(r2)

        with mlflow.start_run() as run:
            # Salvar o modelo final com todos os dados
            modelo_final = xgb.XGBRegressor(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=3,
                random_state=42,
                n_jobs=-1
            )

            modelo_final.fit(X, y)

            mae = np.mean(maes)
            rmse = np.mean(rmses)
            nrmse = np.mean(nrmses)
            r2 = np.mean(r2s)

            # Log no MLflow
            mlflow.log_params({
                  'n_estimators': 100
                , 'learning_rate': 0.05
                , 'max_depth': 3
                , 'random_state': 42
                , 'n_jobs': -1
            })

            mlflow.log_metrics({
                  'mae': mae
                , 'rmse': rmse
                , 'nrmse': nrmse
                , 'r2': r2
            })

            # Inferir a assinatura do modelo
            signature = infer_signature(X, y)

            # Log do modelo com registro direto
            mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path="model",
                signature=signature,
                input_example=X[:5],  # usar amostra pequena
                registered_model_name=model_name
            )

        resultados.append({
            'cluster': cluster,
            'region': region,
            'mae': np.mean(maes),
            'rmse': np.mean(rmses),
            'nrmse': np.mean(nrmses),
            'r2': np.mean(r2s)
        })

    return pd.DataFrame(resultados)


In [4]:
# Excluir colunas que não são features
features = [col for col in df.columns if col not in ['client_id', 'date', 'consumption_kwh', 'cluster', 'region','humidity','temperature']]

# Treinar modelos
resultados = treinar_modelos_xgb_por_grupo_cv(df, features)

# Exibir resultados
print(resultados.sort_values(by='nrmse'))


2025/07/29 23:41:56 INFO mlflow.tracking.fluent: Experiment with name 'previsao_cliente_Leste_0' does not exist. Creating a new experiment.



Treinando modelo para Cluster 0 - Região Leste


c:\Users\JoaoGuelfi\OneDrive - Analytics2Go\Documentos\consumo_energetico\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/29 23:41:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'previsao_cliente_Leste_0'.
2025/07/29 23:42:07 INFO m

🏃 View run bold-deer-305 at: http://127.0.0.1:8080/#/experiments/181723807580388407/runs/3fb12651ae4f40dbbdfbc8aecb65a22a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/181723807580388407

Treinando modelo para Cluster 0 - Região Norte


c:\Users\JoaoGuelfi\OneDrive - Analytics2Go\Documentos\consumo_energetico\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/29 23:42:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'previsao_cliente_Norte_0'.
2025/07/29 23:42:14 INFO m

...

🏃 View run victorious-mouse-822 at: http://127.0.0.1:8080/#/experiments/165097559475770323/runs/56f083a28f804326afe387d361886b8b
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/165097559475770323

Treinando modelo para Cluster 4 - Região Centro


c:\Users\JoaoGuelfi\OneDrive - Analytics2Go\Documentos\consumo_energetico\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/29 23:42:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'previsao_cliente_Centro_4'.
2025/07/29 23:42:58 INFO 

🏃 View run colorful-loon-526 at: http://127.0.0.1:8080/#/experiments/353735827113454424/runs/df9c193cc9cc4f37ac308bb02c868632
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/353735827113454424

Treinando modelo para Cluster 4 - Região Oeste


c:\Users\JoaoGuelfi\OneDrive - Analytics2Go\Documentos\consumo_energetico\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/29 23:42:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'previsao_cliente_Oeste_4'.
2025/07/29 23:43:06 INFO m

🏃 View run bold-ram-623 at: http://127.0.0.1:8080/#/experiments/810257922335493444/runs/34f8719ada3d4300a0fb42d0f3519fee
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/810257922335493444
   cluster  region       mae      rmse     nrmse        r2
4        2   Leste  1.861938  2.313929  0.127730  0.117633
3        1   Oeste  1.875786  2.331130  0.128934 -0.045105
2        1  Centro  1.933846  2.381785  0.136966  0.003101
5        2   Norte  1.931608  2.411411  0.141453  0.150073
6        3     Sul  1.929341  2.424028  0.165003  0.571880
7        4  Centro  1.858881  2.342252  0.179350  0.244100
8        4   Oeste  1.896679  2.409500  0.183050  0.273973
0        0   Leste  1.885971  2.347685  0.199208  0.277638
1        0   Norte  1.858294  2.321190  0.199500  0.255714


In [5]:
con.execute("""
CREATE TABLE IF NOT EXISTS output.previsao_consumo_metricas (
    cluster INTEGER,
    region VARCHAR,
    mae DOUBLE,
    rmse DOUBLE,
    nrmse DOUBLE,
    r2 DOUBLE,
)
""")

In [6]:
# Limpa dados se as tabelas já existirem
con.execute("DELETE FROM output.previsao_consumo_metricas")

In [7]:
# Registra como tabelas temporárias
con.register("resultados", resultados)

In [8]:
# Insere os dados nas tabelas gold
con.execute("INSERT INTO output.previsao_consumo_metricas SELECT * FROM resultados")

In [9]:
con.close()